<a href="https://colab.research.google.com/github/rwcitek/StructureAndInterpretationOfComputerShells-Nushell/blob/rwc/nushell_in_Jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to install nushell in Jupyter

These commands have been tested in Google's CoLab and in GitHub's CodeSpaces, but may work in other Jupyter environments as well.

Display some info about who we are.

In [ ]:
%%script bash
whoami
id
echo ${HOME}

Install some tools in the environment

In [ ]:
%%capture output
%%script bash
export DEBIAN_FRONTEND=noninteractive
sudo apt-get update
sudo apt-get install -y pkg-config libssl-dev jq tree vim curl less elinks


Install and configure the latest nushell.

In [ ]:
%%capture output
%%script bash
cd /usr/local/bin/
elinks --dump https://github.com/nushell/nushell/releases |
  grep -m1 -o https:.*x86_64-unknown-linux-gnu.tar.gz |
  xargs curl -Ls |
  sudo tar -xzf -
sudo ln -snf ./nu-*-unknown-linux-gnu/nu
yes | nu >& /dev/null


Display the command options.  Unfortunately, only some work.

In [ ]:
!nu --help

Display the version

In [ ]:
!nu --version

Get rid of the ASCII art for table display.


In [ ]:
%%script bash
sed -i -e '/mode.*rounded/ { s/mode/# mode/ ; s/$/\n    mode: none / } ' \
  ${HOME}/.config/nushell/config.nu

Verify the "table" setting has been modified in the config file.

In [ ]:
!grep -C3 'mode: none' ${HOME}/.config/nushell/config.nu

## Hello, world.

In [ ]:
!nu -c 'echo "Hello, world"'

The current date and time.

In [ ]:
!nu -c 'date now'

Try piping commands into nushell.

In [ ]:
%%script bash
# doesn't work
echo 'date now' | nu

In [ ]:
%%script bash
# doesn't work
echo 'date now' | nu --stdin

Saving commands to a file and then running to a file does work.

In [ ]:
%%script bash
# does work
echo 'date now' > /tmp/nushell.nu
nu /tmp/nushell.nu



## Workaround for piping

Create a script that saves stdin to a script then have nushell run the script.

In [ ]:
%%script bash
cat <<'eof' | sudo tee /usr/local/bin/nushell > /dev/null
#!/usr/bin/env bash
cat > /tmp/nushell.tmp.nu
nu --config ${HOME}/.config/nushell/config.nu /tmp/nushell.tmp.nu
eof
sudo chmod +x /usr/local/bin/nushell


In [ ]:
%%script nushell
ls | where type == dir

In [ ]:
%%script nushell
echo "Hello, world"
